In [1]:
import numpy as np
from openvino.runtime import Core
import cv2
import os


captured_frames_dir = "captured_frames"
preprocessed_frames_dir = "preprocessed_frames"
os.makedirs(captured_frames_dir, exist_ok=True)
os.makedirs(preprocessed_frames_dir, exist_ok=True)

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image from {image_path}")
        return None

    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

   
    resized_image = cv2.resize(image_rgb, (1280, 768))

    
    normalized_image = resized_image.astype(np.float32) / 255.0

    
    batched_image = np.expand_dims(normalized_image, axis=0)

   
    preprocessed_image_path = os.path.join(preprocessed_frames_dir, "preprocessed_frame.png")
    cv2.imwrite(preprocessed_image_path, (resized_image * 255).astype(np.uint8))  # Save normalized back to uint8

    return batched_image

def perform_ocr(image):
   
    ie = Core()

    
    model_path = r"C:\Users\Arjun\Dhanvantri\intel\text-detection-0004\FP32\text-detection-0004.xml"
    weights_path = r"C:\Users\Arjun\Dhanvantri\intel\text-detection-0004\FP32\text-detection-0004.bin"

   
    net = ie.read_model(model=model_path, weights=weights_path)

    
    compiled_model = ie.compile_model(model=net, device_name="CPU")

    input_layer = compiled_model.input(0)
    output_layer = compiled_model.output(0)

    
    result = compiled_model([image])[output_layer]

    
    text = result[0]  

    return text

def live_scan_document():
    
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

       
        cv2.imshow("Live Document Scan", frame)

       
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

   
    cap.release()
    cv2.destroyAllWindows()

    if ret:
        
        captured_frame_path = os.path.join(captured_frames_dir, "captured_frame.png")
        cv2.imwrite(captured_frame_path, frame)  # Save captured frame
        return captured_frame_path
    else:
        print("Failed to capture the document")
        return None
    
captured_image_path = live_scan_document()
if captured_image_path:
    preprocessed_frame = preprocess_image(captured_image_path)
    if preprocessed_frame is not None:
        ocr_result = perform_ocr(preprocessed_frame)
        print("OCR Result:", ocr_result)
    else:
        print("Preprocessing failed.")
else:
    print("Image capture failed.")


In [ ]:
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearnex import patch_sklearn
import json
import cv2

patch_sklearn()

def preprocess_image(image_path, output_folder):
    """Preprocess the captured image."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image from {image_path}")
        return None

    # Resize and normalize the image
    resized_image = cv2.resize(image, (1280, 768))
    normalized_image = resized_image.astype(np.float32) / 255.0

    # Create output path for the preprocessed image
    preprocessed_image_path = os.path.join(output_folder, "preprocessed_image.png")
    cv2.imwrite(preprocessed_image_path, resized_image)  # Save preprocessed image for debugging

    return normalized_image

def predict_condition_and_find_specialist(medical_condition):
    
    # Check if the input medical_condition is an empty array or list
    if isinstance(medical_condition, (list, np.ndarray)):
        if len(medical_condition) == 0:
            print("No medical condition provided.")
            return "General Physician"
        else:
            # Convert array or list to a string (use the first element if necessary)
            medical_condition = str(medical_condition[0]) if isinstance(medical_condition, (list, np.ndarray)) else str(medical_condition)
    elif not medical_condition:
        print("No medical condition provided.")
        return "General Physician"

    try:
        with open('Medical_Cases.json', 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        print("Medical_Cases.json file not found.")
        return "General Physician"
    except json.JSONDecodeError:
        print("Error reading Medical_Cases.json.")
        return "General Physician"

    # Extract cases and labels
    medical_cases = data.get('cases', [])
    labels = data.get('specialists', [])

    # Ensure cases and labels are not empty
    if len(medical_cases) == 0 or len(labels) == 0:
        print("Medical cases or labels are empty.")
        return "General Physician"

    # Vectorize the text data
    vectorizer = CountVectorizer()
    X_cases = vectorizer.fit_transform(medical_cases)

    # Encode labels
    le = LabelEncoder()
    y_labels = le.fit_transform(labels)

    # Logistic regression classifier
    clf = LogisticRegression(max_iter=1000)  # Increase max_iter to avoid convergence warnings
    clf.fit(X_cases, y_labels)

    def predict_condition(extracted_text):
        # Ensure extracted_text is a string
        if isinstance(extracted_text, np.ndarray):
            extracted_text = ' '.join(map(str, extracted_text.flatten()))  # Convert to space-separated string

        # Transform the input text to match the vectorizer format
        X_text = vectorizer.transform([extracted_text])

        # Get the prediction, and ensure it's a single value
        prediction = clf.predict(X_text)

        # Ensure prediction is not empty
        if prediction.size == 0:
            print("No prediction made.")
            return "General Physician"

        # Use the first prediction only
        predicted_label = prediction[0]

        # Return the inverse-transformed predicted label
        return le.inverse_transform([predicted_label])[0]

    # Predict the specialist based on the medical condition
    predicted_specialist = predict_condition(medical_condition)
    print(f"Predicted Specialist: {predicted_specialist}")

    return predicted_specialist


# Example usage
# Set up directories for captured and preprocessed images
captured_image_folder = "captured_frames"  # Folder where captured images are stored
preprocessed_image_folder = "preprocessed_frames"  # Folder for preprocessed images

# Ensure folders exist
os.makedirs(captured_image_folder, exist_ok=True)
os.makedirs(preprocessed_image_folder, exist_ok=True)

# Simulate image capture and preprocessing
captured_image_path = os.path.join(captured_image_folder, "captured_frame.png")  # Example captured image path
preprocessed_image = preprocess_image(captured_image_path, preprocessed_image_folder)

# Assume 'ocr_result' variable is defined elsewhere in your code
if 'ocr_result' in globals():  
    medical_condition = ocr_result  # Use the OCR result if available
    # Ensure medical_condition is a string
    if isinstance(medical_condition, (np.ndarray, list)):
        medical_condition = ' '.join(map(str, medical_condition))  # Convert all elements to strings and join
else:
    medical_condition = "diabetes"  # Fallback to a default example condition

In [3]:
import pandas as pd
import json
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

def get_current_location():
    address = input("Enter your current address: ")
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(address)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        print("Address not found. Please try again.")
        return get_current_location()

def recommend_nearby_hospitals(current_location, max_distance_km=25):
   
    with open('Hospitals.json', 'r') as file:
        data = json.load(file)

    
    hospitals_series = pd.Series(data['Sheet1'])

    recommendations = []

    
    for hospital in hospitals_series:
        latitude = hospital.get('Latitude')
        longitude = hospital.get('Longitude')

        if latitude is not None and longitude is not None:
            hospital_location = (float(latitude), float(longitude))

            
            distance_km = geodesic(current_location, hospital_location).kilometers

           
            if distance_km <= max_distance_km:
                recommendations.append({
                    'Hospital Name': hospital.get('Hospital Name', 'N/A'),
                    'City': hospital.get('City', 'N/A'),
                    'Rating': hospital.get('Rating', 'N/A'),
                    'Distance (km)': round(distance_km, 2)
                })

    if recommendations:
        print(f"Hospitals within {max_distance_km} km of your location:")
        for hospital in recommendations:
            print(f"- {hospital['Hospital Name']} ({hospital['City']}), Rating: {hospital['Rating']}, Distance: {hospital['Distance (km)']} km")
    else:
        print(f"No hospitals found within {max_distance_km} km.")

In [ ]:
specialist = predict_condition_and_find_specialist(medical_condition) # Specialist recommendation based on condition of the patient
print(f"Recommended Specialist: {specialist}")

current_location = get_current_location() # Hospitals nearby to the patient
recommend_nearby_hospitals(current_location)